In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime

import os
import json
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


Create a folder for measurement data

In [2]:
now = datetime.now()
path = adress +'\\'+ now.strftime('%Y-%m-%d')
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2023-09-15 already exists


In [3]:
try: print(vna.query('*IDN?'))
except: 
    vna = RohdeSchwarzZNB8('VNA', 'GPIB0::20::INSTR', init_s_params=False, reset_channels=False)
    vna.add_channel('S21')
    vna.display_single_window()
    vna.channels.autoscale()
    vna.channels.S21.setup_cw_sweep()

def setup_vna_CW(cw_frequency=6e9, vna_power=-20, npts=10, bandwidth=10):
    vna.channels.S21.power(vna_power)
    vna.channels.S21.npts(npts)
    vna.channels.S21.bandwidth(bandwidth)
    vna.channels.S21._set_cw_frequency(cw_frequency)
    vna.rf_on()
    I, Q = vna.channels.S21._get_cw_data()
    vna.rf_off()
    S=I+1j*Q
    return abs(S), np.angle(S)

Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062102129, firmware:3.45) in 0.08s


In [4]:
try: print(anapico.query('*IDN?'))
except: 
    rm = pyvisa.ResourceManager()
#     anapico = rm.open_resource('USB0::0x03EB::0xAFFF::3C6-0B4F40003-0985::INSTR') # USB connection
    anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
    print(anapico.query('*IDN?'))


def setup_anapico(power=-30, frequency = 6e9, channel = 1):
    if flux_power<=-6:
        print('Flux power set:', power)
        anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))
        anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(frequency)))
        return True
    else: 
        print('Flux power is too high:', flux_power)
        return False

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [5]:
flux_powers = np.linspace(-30, -6, 20)
# flux_powers = np.linspace(-10, -6, 1)

flux_freq = 11.252e9
avg_total = 200

vna_freq = 5.78195e9
vna_bandwidth = 100
number_of_points = int(vna.ask('SENS1:SWE:POIN?'))
vna_power = 0

In [56]:
anapico.write(r'SOUR1:POW -30')
anapico.write(r'OUTP1 OFF')
data = dict.fromkeys(flux_powers, None)
for ip, flux_power in enumerate(flux_powers):
    anapico.write(r'OUTP1 ON')
    status = setup_anapico(power=flux_power, frequency = flux_freq)
    if status:
        fft_avg_mag = np.zeros(number_of_points)
        fft_avg_pha = np.zeros(number_of_points)
        for avg in range(avg_total):
            mag, pha = setup_vna_CW(vna_freq, vna_power, number_of_points, vna_bandwidth)
            fft_avg_mag+=fft_A(mag)
            fft_avg_pha+=fft_A(pha)
            
        data[flux_power] = np.array([[fft_avg_mag/avg_total], [fft_avg_pha/avg_total]])
        
anapico.write(r'OUTP1 OFF')
# anapico.close()

Flux power set: -30.0
Flux power set: -28.736842105263158
Flux power set: -27.473684210526315
Flux power set: -26.210526315789473
Flux power set: -24.94736842105263
Flux power set: -23.684210526315788
Flux power set: -22.42105263157895
Flux power set: -21.157894736842106
Flux power set: -19.894736842105264
Flux power set: -18.63157894736842
Flux power set: -17.36842105263158
Flux power set: -16.10526315789474
Flux power set: -14.842105263157896
Flux power set: -13.578947368421055
Flux power set: -12.315789473684212
Flux power set: -11.05263157894737
Flux power set: -9.789473684210527
Flux power set: -8.526315789473685
Flux power set: -7.2631578947368425
Flux power set: -6.0


11

In [6]:
%matplotlib qt
for i in range(len(flux_powers)):
    plt.loglog(fft_f(), data[flux_powers[i]][1][0])

NameError: name 'fft_f' is not defined

In [73]:
ind = [0, 4,  8, 10, 12, 14, 16, 17, 18, 19]
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
for i in ind:
    ax[0].loglog(fft_f(), data[flux_powers[i]][1][0], label = '{}'.format(round(flux_powers[i],2)))
    ax[1].loglog(fft_f(), data[flux_powers[i]][0][0], label = '{}'.format(round(flux_powers[i],2)))
    
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')
ax[0].legend()
ax[1].legend()

In [48]:
d3 = data[flux_powers[0]][1][0]

In [44]:
d2 = data[flux_powers[0]][1][0]

In [40]:
d1 = data[flux_powers[0]][1][0]

In [58]:
# flux_powers = np.linspace(-30, -6, 20)
flux_powers = -6

flux_freq = 11.252e9
avg_total = 200

vna_freq = 5.78195e9
vna_bandwidth = 100
number_of_points = int(vna.ask('SENS1:SWE:POIN?'))
vna_powers = np.linspace(-30, 0, 20)

In [59]:
anapico.write(r'SOUR1:POW -30')
anapico.write(r'OUTP1 OFF')
data_vna_sweep = dict.fromkeys(flux_powers, None)
for ip, vna_power in enumerate(vna_powers):
    anapico.write(r'OUTP1 ON')
    status = setup_anapico(power=flux_powers, frequency = flux_freq)
    if status:
        fft_avg_mag = np.zeros(number_of_points)
        fft_avg_pha = np.zeros(number_of_points)
        for avg in range(avg_total):
            mag, pha = setup_vna_CW(vna_freq, vna_power, number_of_points, vna_bandwidth)
            fft_avg_mag+=fft_A(mag)
            fft_avg_pha+=fft_A(pha)
            
        data_vna_sweep[flux_power] = np.array([[fft_avg_mag/avg_total], [fft_avg_pha/avg_total]])
        
anapico.write(r'OUTP1 OFF')
# anapico.close()

TypeError: 'int' object is not iterable

In [ ]:
plt.loglog(fft_f(), data_vna_sweep[vna_freq[:]][1][0])

In [33]:
def fft_A(A):
    fft_A = abs(fftshift(fft(A))**2)
    return fft_A

def fft_f(bandwidth=vna_bandwidth, nop=number_of_points):
    f = np.linspace(-bandwidth/2, bandwidth/2, nop)
    return f

In [9]:
%matplotlib qt
for power in flux_powers:
    selected_data = data[power][1]
    mag = np.zeros(len(selected_data[0][0,:]))
    pha = np.zeros(len(selected_data[0][0,:]))
    M = np.array([])
    P = np.array([])
    for i in range(avg_total):
        mag1 = selected_data[i][0,0,:]
        pha1 = selected_data[i][1,0,:]
        mag = mag+mag1
        pha=pha+pha1

#     f, A =  plot_noise(mag)
#     plt.loglog(f, A)
    f, A =  plot_noise(pha)
    plt.loglog(f, A)

In [76]:
data_to_save=np.empty((len(flux_powers),2,number_of_points))
for i in range(len(flux_powers)):
    for j in range(2):
        for k in range(number_of_points): 
            data_to_save[i,j,k] = data[flux_powers[i]][j][0][k]

In [79]:
plt.loglog(fft_f(), data_to_save[-2,1,:])

In [80]:
dict_data=dict()
dict_data['flux_powers'] = flux_powers
dict_data['data'] = data_to_save
dict_data['freqs'] = fft_f()

experiment_name = 'Saving_test'
file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
savemat(file_name, dict_data)

Create log file

In [81]:
comment = 'Power Sweep for TWPA for different flux powers and fixed VNA powers'


file_name = adress +'\\'+'log_'+exp_name+'.txt'
# f = open(file_name, 'w+')  # open if not exist
f = open(file_name, 'a')  # append  to existing file
f.write(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))+':   ')
f.write(comment+'\n')
f.close()